In [ ]:
import pandas as pd
import networkx as nx
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from scipy.special import comb
import seaborn as sns
import numpy as np
import sys
from pylab import rcParams
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import skfda
from collections import Counter
import pygraphviz as pgv
from pyvis.network import Network
import matplotlib.colors as pltcol
from scipy.cluster.hierarchy import dendrogram, linkage, set_link_color_palette, fcluster
from scipy.spatial.distance import squareform
from sklearn import preprocessing
from matplotlib.pyplot import figure
from IPython.display import display

In [ ]:
colors = [plt.cm.Set3(i) for i in range(20)]
high_regime=colors[0]
low_regime=colors[3]
default_color1=colors[4]
default_color2=low_regime
default_color3=high_regime
default_color4=colors[5]
plt.cm.get_cmap('Set3')

In [ ]:
#from https://github.com/JohnDeJesus22/DataScienceMathFunctions/blob/master/hypergeometricfunctions.py
def hypergeom_pmf(N, A, n, x):
    
    '''
    Probability Mass Function for Hypergeometric Distribution
    :param N: population size
    :param A: total number of desired items in N
    :param n: number of draws made from N
    :param x: number of desired items in our draw of n items
    :returns: PMF computed at x
    '''
    Achoosex = comb(A,x)
    NAchoosenx = comb(N-A, n-x)
    Nchoosen = comb(N,n)
    
    return (Achoosex)*NAchoosenx/Nchoosen

def hypergeom_cdf(N, A, n, t, min_value=None):
    
    '''
    Cumulative Density Funtion for Hypergeometric Distribution
    :param N: population size
    :param A: total number of desired items in N
    :param n: number of draws made from N
    :param t: number of desired items in our draw of n items up to t
    :returns: CDF computed up to t
    '''
    if min_value:
        return np.sum([hypergeom_pmf(N, A, n, x) for x in range(min_value, t+1)])
    
    return np.sum([hypergeom_pmf(N, A, n, x) for x in range(t+1)])

def get_succ(status):
    if status == 'Acquired':
        return 1
    if status == 'Alive / Active':
        return 0
    if status == 'IPO / Went public':
        return 1
    if status == 'Merged':
        return 1
    if isinstance(status, float):
        if np.isnan(status):
            return np.nan
    else:
        return 0
    
    def get_status(rounds):
    a = list()
    for i in rounds:
        indice = stage.index(i)
        a.append(stato[indice])
    return a
    
    


In [ ]:
cb1 = pd.read_csv('cb1.csv')
cb2 = pd.read_csv('cb2.csv')
cb3 = pd.read_csv('cb3.csv')
cb = cb1.append(cb2)
cb = cb.append(cb3)
cb.index = range(len(cb))
cb['date_my'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('m')
cb['date_y'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('y')

nodes_org = cb[['target_name', 'target_ID', 'target_country', 'target_continent', 'industry', 'target_founded','target_status']].drop_duplicates()
nodes_org['bipartite'] = 1
nodes_inv = cb[['investors_name', 'investor_id', 'investor_country', 'investor_continent', 'investor_type', 'investor_founded']].drop_duplicates()
nodes_org = nodes_org.rename(columns={"target_name": "node", "target_country": "country", 
                                      "target_continent": "continent", "industry":"industry", "target_status":"status",
                                     "target_ID":"index", 'target_founded':'foundation'})
nodes_inv = nodes_inv.rename(columns={"investors_name": "node","investor_country": "country", 
                                      "investor_continent": "continent", "investor_type":"industry","investor_id":'index', 'investor_founded':'foundation' })
nodes_inv['bipartite'] = 0
nodes = nodes_org.append(nodes_inv, ignore_index= True)
nodes['id'] = nodes.index

numbers = cb.groupby(['target_name','stage'], as_index = False).count()[['target_name','stage','investors_name']].rename(columns={"investors_name": "investor_numbers"})
cb_new = pd.merge(cb, numbers,  how='left', left_on=['target_name','stage'], right_on = ['target_name','stage'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 1],  how='left', left_on=['target_name','target_ID'], right_on = ['node','index'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 0],  how='left', left_on=['investors_name','investor_id'], right_on = ['node','index'])


In [ ]:
stati = cb_new.groupby('id_x')
stati2 = stati.tail(1).reset_index(drop=True)
stati3 = stati2[stati2["date"].notna()]

fun_status = stati2[["round_simp", "target_status"]].groupby(['round_simp', "target_status"]).size().unstack(fill_value=0)[1:61]

stage = list(fun_status.idxmax(axis=1).index)
stage.append("Acq - P2P")

stato = list(fun_status.idxmax(axis=1))
stato.append("Acquired")

set(stato)
#robustness nel passare degli anni

In [ ]:
df = cb_new
df['date_y']=df['date_y'].dt.year
df = df[df['date_y']>0]
ids = set(df['id_x'])

company = list(ids)

for i in range(5,13):
    locals()['status'+ str(i)] = []

for i in tqdm(ids):
    sub = df[df['id_x']== i][['id_x', 'date_y', 'round_simp','target_founded']]
    for l in range(5,13):
        sub1 = sub[(sub['date_y']<sub['date_y'].values[0]+l) & (sub['date_y'].values[0] < 2022-l)]
        sub2 = sub1.tail(1)
        #company.append(i)
        roun = sub2['round_simp']
        locals()['status'+ str(l)].append(get_status(roun)) 
    if i % 10000 ==0:
        print(i)


In [ ]:
for i in range(5,13):
    locals()['stati'+str(i)] = [x[0] if len(x) >= 1 else np.nan for x in locals()['status'+ str(i)]]

In [ ]:
dicts = {}
dicts['id']=company
for i in range(5,13):
        dicts['st'+str(i)] = locals()['stati'+str(i)]
df_stati = pd.DataFrame(dicts)

In [ ]:
covariates = pd.read_csv('covariates.csv')
bonav = covariates

In [ ]:
succ = []
for i in range(0, len(bonav)):
    succ.append(get_succ(bonav.iloc[i,:]['current_sit']))
outcome = list(set(bonav['current_sit']))
success = [get_succ(x) for x in outcome]

df_s = pd.DataFrame({'current_sit': outcome, 'success' : success})
bonav2 = pd.merge(bonav, df_s, on = 'current_sit')
bonav3 = pd.merge(bonav2, df_stati, on = 'id')

for l in range(5,13):
    locals()['succ'+str(l)]= []
    for i in tqdm(range(0,len(bonav3))):
        locals()['succ'+str(l)].append(get_succ(bonav3.iloc[i,:]['st' +str(l)]))

In [ ]:
for i in range(5,13):
    bonav3['succ'+str(i)] = locals()['succ'+str(i)]

In [ ]:
#FISSO AMPIEZZA SAMPLE, MUOVO ANNO INVESTIMENTO

covars = list(Z_scaled.columns) #take the centralities from the list of covariates 
clus = list(fl) #retrieve the clusters found by the dendrogram
diz = {'covars':covars, 'clus':clus}
clus_n = range(1,8)
dataf = pd.DataFrame(diz)

groups = [1,2,3,4,5,6,7]
colours = [0,2,3,5,6,9,7]
yr = 10
x = []
y = []
z = []
c = []
err = []
for l in groups:
    cents = list(dataf[dataf['clus']==l]['covars'])
    print(l)
    for cent in cents:
        b12 = bonav3
        b12 = b12[[cent, 'current_sit','success','stage', 'st8', 'succ'+str(yr),'id', 'node','date']]
        b12 = b12[b12[cent].notnull()]
        b12 = b12[b12['succ'+str(yr)].notnull()]
        moves = []
        p = []
        ran = []
        std = []
        for i in range(2000,2022-yr):
            n = 25
            N = len(b12[b12['date']==i])
            A = int(sum(b12[b12['date']==i]['succ'+str(yr)]))
            ran.append(A/N)
            if cent in ['voterank_median','voterank_max','voterank_min']:
                t = int(sum(b12[b12['date']==i].sort_values(ascending=True, by = cent)[0:n]['succ'+str(yr)]))
            else:
                t = int(sum(b12[b12['date']==i].sort_values(ascending=False, by = cent)[0:n]['succ'+str(yr)]))
            moves.append(t/n)
            p.append(hypergeom_cdf(N, A, n, t, min_value=None))
            std.append(np.sqrt(n*A/N*(1-A/N)*(N-n)/np.sqrt((N-1))))
        c.append(colours[l-1])
        x.append(np.mean(moves))
        y.append(cent)
        z.append(np.mean(ran))
        err.append(np.std(moves))
        std = np.mean(std)/len(std)
colors = [plt.cm.Set3(i) for i in range(20)]
x.append(z[0])
y.append('random')
c.append('b')
err.append(std)
df = pd.DataFrame(list(zip(x, y, c, err)), columns =['rate', 'cent', 'color', 'err'])
df2 = df.sort_values(by=['rate'], ascending = False)
c = df2.color
c2 = []
c3 = []
for i in c:
    if i != 'b':
        c2.append(colors[i])
    else:
        c2.append('k')
for i in c:
    if i != 'b':
        c3.append('k')
    else:
        c3.append('r')
        
y= list(df2.cent)
x= list(df2.rate)
err = list(df2.err)

fig = plt.figure(figsize=(12, 12))    
container = plt.barh(y, x, xerr = err, align='center', color = c2, capsize=3)
plt.ylabel("Centralities")
connector, caplines, (horizontal_lines,) = container.errorbar.lines
horizontal_lines.set_color(c3)
plt.xlabel("Success rate")
plt.tight_layout()
plt.savefig('bonav' +'.pdf')

plt.show()